In [ ]:
from keras.models import Sequential
from keras.layers import Dense
import keras
import pandas as pd
import numpy as np
from keras.models import load_model
from keras.layers import Dropout
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
path = "/content/wd.csv"
df = pd.read_csv(path,sep=",")

In [ ]:
df.drop(labels=["Unnamed: 0"],axis=1,inplace=True)

In [ ]:
data = df.iloc[:-1,1:].values
data = np.transpose(data)
total = df.iloc[-1][1:].values
X = np.asarray(data).astype(np.float32)
Y = np.asarray(total).astype(np.float32)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, total, test_size = 1/5, random_state = 123, shuffle=1)
X = np.asarray(X_train).astype(np.float32)
Y = np.asarray(y_train).astype(np.float32)
X_test = np.asarray(X_test).astype(np.float32)
Y_test = np.asarray(y_test).astype(np.float32)

In [ ]:
model = Sequential()
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(1))
model.compile(optimizer='Adam',
              loss='mean_absolute_error',
              metrics=['RootMeanSquaredError'])
history = model.fit(X,Y,
          batch_size=16,
          epochs=5000,
          verbose=1)
model.summary()

In [ ]:
predict = model.predict(X)
for i in range(len(X)):
  print(int(abs(predict[i]-Y[i])))

In [ ]:
plt.plot(history.history['loss'])
#plt.plot(history.history['val_loss'])
plt.title('Mean Absolute Error')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
#model = keras.models.load_model('path/to/location.keras')

In [ ]:
model.save('model.keras')
def convert(model):
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    tflite_model = converter.convert()

    with open('app_model.tflite', 'wb') as f:
        f.write(tflite_model)
convert(model)